## Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [2]:
 #https://www.google.com/url?q=https%3A%2F%2Fs3.amazonaws.com%2Fdl4j-distribution%2FGoogleNews-vectors-negative300.bin.gz&sa=D&sntz=1&usg=AFQjCNGezmGpab4ndAoju5CHhdYRNPDD8w

In [3]:
data = pd.read_csv(r"C:\Users\Manali\Downloads\Hellbnet\offensive language\agr_en_train.csv")
data.head()

,Unique_id,Text,Aggression_level
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG


In [4]:
data=data.drop(["Unique_id"],axis=1)
data.head()

,Text,Aggression_level
0,Well said sonu..you have courage to stand agai...,OAG
1,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,"Now question is, Pakistan will adhere to this?",OAG
3,Pakistan is comprised of fake muslims who does...,OAG
4,"??we r against cow slaughter,so of course it w...",NAG


In [5]:
data.Aggression_level[data.Aggression_level == 'OAG']  = 2
data.Aggression_level[data.Aggression_level == 'NAG']  = 0
data.Aggression_level[data.Aggression_level == 'CAG']  = 1

In [6]:
data.head()

,Text,Aggression_level
0,Well said sonu..you have courage to stand agai...,2
1,"Most of Private Banks ATM's Like HDFC, ICICI e...",0
2,"Now question is, Pakistan will adhere to this?",2
3,Pakistan is comprised of fake muslims who does...,2
4,"??we r against cow slaughter,so of course it w...",0


In [7]:
data.Aggression_level.value_counts()

0    5051
1    4240
2    2708
Name: Aggression_level, dtype: int64

In [8]:
text = data[["Text"]]
Aggression_level = data[["Aggression_level"]]
Aggression_level = data.query("Aggression_level == 'OAG'")[["Aggression_level"]]
Aggression_level = data.query("Aggression_level == 'NAG'")[["Aggression_level"]]
Aggression_level = data.query("Aggression_level == 'CAG'")[["Aggression_level"]]

clean_tweets = copy.deepcopy(text)

In [9]:
data.describe()

,Text,Aggression_level
count,11999,11999
unique,11643,3
top,Gandhi Killer Ram (Nathu) Temple in India........,0
freq,14,5051


## Preprocessing

In [10]:
#importing neccessarcy modules
import re
import nltk
nltk.download('punkt', 'stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [11]:
def take_data_to_shower(Text):
    noises = [ '\'ve', 'n\'t', '\'s', '\'m' , '?' , ':(' , 'ðŸ˜¬ðŸ˜œ' , "ðŸ˜‚ðŸ˜‚ðŸ˜‚" , 'ðŸ˜Ž' , 'â†’' , 'â‚¹vs $']

    for noise in noises:
        Text = Text.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', Text)


def tokenize(Text):
    lower_tweet = Text.lower()
    return word_tokenize(lower_tweet)


def remove_stop_words(tokens):
    clean_tokens = []
    stopWords = set(stopwords.words('english'))
    for token in tokens:
        if token not in stopWords:
            if token.replace(' ', '') != '':
                if len(token) > 1:
                    clean_tokens.append(token)
    return clean_tokens


def stem_and_lem(tokens):
    clean_tokens = []
    for token in tokens:
        token = wordnet_lemmatizer.lemmatize(token)
        token = lancaster_stemmer.stem(token)
        if len(token) > 1:
            clean_tokens.append(token)
    return clean_tokens

In [12]:
 import nltk
#nltk.download()

In [13]:
tqdm.pandas(desc="Cleaning Data Phase I...")
data['Text'] = data['Text'].progress_apply(take_data_to_shower)

tqdm.pandas(desc="Tokenizing Data...")
data['tokens'] = data['Text'].progress_apply(tokenize)

tqdm.pandas(desc="Cleaning Data Phase II...")
data['tokens'] = data['tokens'].progress_apply(remove_stop_words)

tqdm.pandas(desc="Stemming And Lemmatizing")
data['tokens'] = data['tokens'].progress_apply(stem_and_lem)

text_vector = data['tokens'].tolist()

Stemming And Lemmatizing: 100%|██████████████████████████████████████████████| 11999/11999 [00:13<00:00, 891.48it/s]


In [14]:
data.head()

,Text,Aggression_level,tokens
0,Well said sonu you have courage to stand agai...,2,"[wel, said, sonu, cour, stand, dadagir, muslim]"
1,Most of Private Banks ATM Like HDFC ICICI etc...,0,"[priv, bank, atm, lik, hdfc, icic, etc, cash, ..."
2,Now question is Pakistan will adhere to this,2,"[quest, pak, adh]"
3,Pakistan is comprised of fake muslims who does...,2,"[pak, compr, fak, muslim, know, mean, un, impo..."
4,we r against cow slaughter so of course it wil...,0,"[cow, slaught, cours, stop, leath, manufact, hap]"


In [15]:
for text in range(len(clean_tweets)):
    data["Text"][text]=data["Text"][text].lower()
low_tokens =data.head()

In [16]:
low_tokens

,Text,Aggression_level,tokens
0,well said sonu you have courage to stand agai...,2,"[wel, said, sonu, cour, stand, dadagir, muslim]"
1,most of private banks atm like hdfc icici etc...,0,"[priv, bank, atm, lik, hdfc, icic, etc, cash, ..."
2,now question is pakistan will adhere to this,2,"[quest, pak, adh]"
3,pakistan is comprised of fake muslims who does...,2,"[pak, compr, fak, muslim, know, mean, un, impo..."
4,we r against cow slaughter so of course it wil...,0,"[cow, slaught, cours, stop, leath, manufact, hap]"


In [17]:
binaryOP = []
for i in data.Aggression_level:
    if i==0:
        binaryOP.append([1,0,0])
    elif i==1:
        binaryOP.append([0,1,0])
    elif i==2:
        binaryOP.append([0,0,1])

print(binaryOP)

binaryOpNP = np.array(binaryOP)

[[0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0]

In [18]:
data['binary_label']= binaryOP


data = data[['Text','tokens',  'Aggression_level', 'binary_label']]
data.head()
    

,Text,tokens,Aggression_level,binary_label
0,well said sonu you have courage to stand agai...,"[wel, said, sonu, cour, stand, dadagir, muslim]",2,"[0, 0, 1]"
1,most of private banks atm like hdfc icici etc...,"[priv, bank, atm, lik, hdfc, icic, etc, cash, ...",0,"[1, 0, 0]"
2,now question is pakistan will adhere to this,"[quest, pak, adh]",2,"[0, 0, 1]"
3,pakistan is comprised of fake muslims who does...,"[pak, compr, fak, muslim, know, mean, un, impo...",2,"[0, 0, 1]"
4,we r against cow slaughter so of course it wil...,"[cow, slaught, cours, stop, leath, manufact, hap]",0,"[1, 0, 0]"


In [19]:
from sklearn.model_selection import train_test_split                                          

data_train, data_test = train_test_split(data, 
                                          test_size=0.10)
data_train.head()

,Text,tokens,Aggression_level,binary_label
5689,prabhu inefficiency is the reality cant hid...,"[prabhu, inefficy, real, cant, hid]",1,"[0, 1, 0]"
929,mfuk who has done dis i know indian express ...,"[mfuk, don, dis, know, ind, express, backgroun...",2,"[0, 0, 1]"
7936,so good to know bahubali has done super bus...,"[good, know, bahubal, don, sup, busy, us]",0,"[1, 0, 0]"
9181,these are the repercussions of making an unedu...,"[repercuss, mak, uneduc, person, pm, stil, lot...",2,"[0, 0, 1]"
8726,bot user xingh kannu python bot team ...,"[bot, us, xingh, kannu, python, bot, team, tay...",0,"[1, 0, 0]"


# Embedding

In [20]:
import gensim

model = gensim.models.Word2Vec(sentences=data['tokens'], size=200, window=5, workers=4, min_count=1)

words = list(model.wv.vocab)
print("Vocabulary size: " + str(len(words)))

filename = 'modelWord.txt'
model.wv.save_word2vec_format(filename, binary=False)


import os

#loadinig the model 

embeddings_index = {}
f = open(os.path.join('','modelWord.txt',), encoding = "utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:])
    embeddings_index[word] = coef

f.close()



Vocabulary size: 12643


In [21]:
#vectorizing the samples  

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot

objTokenizer = Tokenizer()
objTokenizer.fit_on_texts(finalProcessedText)
sequences = objTokenizer.texts_to_sequences(finalProcessedText)

word_index = objTokenizer.word_index
print("unique tokens : " + str(len(word_index)))

words_array = pad_sequences(sequences, maxlen = 200)
print("shape :", words_array.shape)


Using TensorFlow backend.
C:\Users\Manali\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Manali\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Manali\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Manali\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

NameError: name 'finalProcessedText' is not defined

In [58]:
#creating embedding matrix

num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 200))

for word, i in word_index.items():
    if i> num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(num_words)

7


# Model

In [59]:

from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers.core import Activation, Dropout, Dense

from keras.models import Sequential
model = Sequential()

embedding_layer = Embedding(12644, 200, weights=[embedding_matrix], input_length=200 , trainable=False)
model.add(embedding_layer)

model.add(Flatten())

model.add(Dense(3, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

ValueError: Layer weight shape (12644, 200) not compatible with provided weight shape (7, 200)

In [109]:
#Splitting the data set
validation_split = 0.2

labelBinaryFormatNP = binaryOpNP
indices  = np.arange(len(words_array))
np.random.shuffle(indices)

words1 = words_array[indices]
labels1 = labelBinaryFormatNP[indices]

num_validation_sample = int(validation_split * all_sentence.shape[0])

print(num_validation_sample)
X_train = words1[:-num_validation_sample]
y_train = labels1[:-num_validation_sample]

X_test = words1[-num_validation_sample:]
y_test = labels1[-num_validation_sample:]

print(y_train.shape)
print(X_train.shape)

2399
(9600, 3)
(9600, 200)


In [110]:
#fitting the model
model.fit(X_train, y_train, batch_size = 128, epochs = 10, validation_data=(X_test,y_test), verbose =2)

Train on 9600 samples, validate on 2399 samples
Epoch 1/10
 - 2s - loss: 0.6443 - acc: 0.6534 - val_loss: 0.6357 - val_acc: 0.6621
Epoch 2/10
 - 2s - loss: 0.6291 - acc: 0.6614 - val_loss: 0.6330 - val_acc: 0.6653
Epoch 3/10
 - 2s - loss: 0.6239 - acc: 0.6662 - val_loss: 0.6364 - val_acc: 0.6610
Epoch 4/10
 - 2s - loss: 0.6201 - acc: 0.6668 - val_loss: 0.6343 - val_acc: 0.6611
Epoch 5/10
 - 2s - loss: 0.6186 - acc: 0.6691 - val_loss: 0.6316 - val_acc: 0.6635
Epoch 6/10
 - 2s - loss: 0.6166 - acc: 0.6675 - val_loss: 0.6388 - val_acc: 0.6611
Epoch 7/10
 - 2s - loss: 0.6130 - acc: 0.6713 - val_loss: 0.6325 - val_acc: 0.6571
Epoch 8/10
 - 2s - loss: 0.6139 - acc: 0.6708 - val_loss: 0.6343 - val_acc: 0.6685
Epoch 9/10
 - 2s - loss: 0.6145 - acc: 0.6724 - val_loss: 0.6338 - val_acc: 0.6614
Epoch 10/10
 - 2s - loss: 0.6114 - acc: 0.6721 - val_loss: 0.6335 - val_acc: 0.6629


In [111]:
#evaluating the model
evalModel = model.evaluate(X_test, y_test)


2399/2399 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 100us/step


In [114]:

print("accuracy : ", evalModel[1])

accuracy :  0.6629151701927185


In [113]:
from sklearn import metrics
#confusion matrix

yTestCat = []
for i in y_test:
    array = ''
    for j in i:
        array += str(j)
    if(array == "100"):
        yTestCat.append(0)
    
    if(array == "010"):
        yTestCat.append(1)
    
    if(array == "001"):
        yTestCat.append(2)
        
#confusionMatrix
ypred = model.predict_classes(X_test)
CMatrix = metrics.confusion_matrix(yTestCat, ypred) 

print("Confusion Matrix Obtained : ")
print(CMatrix)

Confusion Matrix Obtained : 
[[746 115 122]
 [549 166 143]
 [304 110 144]]
